In [11]:
import numpy as np
#import keras
from keras.layers import Dense, Input, Conv2D, MaxPooling2D, UpSampling2D
#from keras.layers import Activation, Dense, Input
#from keras.layers import , Flatten
#from keras.layers import Reshape, Conv2DTranspose
from keras.models import Model
from keras.datasets import mnist
import keras.backend as K
import matplotlib.pyplot as plt
#from PIL import Image
#from skimage.transform import resize

In [1]:
# Funciones útiles

def noise_data(noise_location, noise_strength):
  (x_train, _), (x_test, _) = mnist.load_data()

  image_size = x_train.shape[1]
  x_train = np.reshape(x_train, [-1, 28,28, 1])/255.0
  x_test = np.reshape(x_test, [-1, 28,28, 1])/255.0
  input_shape = (28,28, 1)

  noise = np.random.normal(loc=noise_location, scale=noise_strength, size=x_train.shape)
  x_train_noisy = x_train + noise
  noise = np.random.normal(loc=noise_location, scale=noise_strength, size=x_test.shape)
  x_test_noisy = x_test + noise

  x_train_noisy = np.clip(x_train_noisy, 0., 1.)
  x_test_noisy = np.clip(x_test_noisy, 0., 1.)
  return x_train,x_train_noisy,x_test,x_test_noisy,input_shape

def show_as_single_image(imgs,image_shape=None):
  if image_shape is None:
    n,h,w,c=imgs.shape
  else:
    h,w=image_shape
    n,c=imgs.shape
  imgs = imgs.reshape((n, h,w))
  imgs = imgs.swapaxes(1,2).reshape((n*w, h)).swapaxes(0,1)
  imgs = (imgs * 255).astype(np.uint8)

  plt.figure(figsize=(1*8,n*8),dpi=100)
  plt.axis('off')
  plt.imshow(imgs, interpolation='none', cmap='gray')
  plt.tight_layout()

In [6]:
# Carga de datos

noise_location=0.3
noise_strength=0.2
x_train,x_train_noisy,x_test,x_test_noisy,input_shape = noise_data(noise_location,noise_strength)

In [ ]:
# Visualización de las imágenes originales y sus versiones ruidosas

num=10 # cantidad de imágenes a visualizar
print("Originales")
show_as_single_image(x_test[:num])
print("Ruidosas")
show_as_single_image(x_test_noisy[:num])

In [ ]:
def ConvolutionalAutoencoder(input_shape):
  filters = 32

  def generate_encoder():
    encoder_input = Input(shape=input_shape, name='encoder_input')

    x = Conv2D(filters, (3, 3), activation='relu', padding='same')(encoder_input)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(filters//2, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(filters//2, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)

    encoded_shape = K.int_shape(x)[1:]

    encoder = Model(encoder_input, x, name='encoder')
    return encoder,encoder_input,encoded_shape

  def generate_decoder(encoded_shape):
    latent_input = Input(shape=encoded_shape, name='decoder_input')

    x = Conv2D(filters//2, (3, 3), activation='relu', padding='same')(latent_input)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(filters//2, (3, 3), activation='relu', padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(filters, (3, 3), activation='relu')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    decoder = Model(latent_input, x, name='decoder')
    return decoder,latent_input

  encoder,encoder_input,encoded_shape = generate_encoder()
  decoder,latent_input = generate_decoder(encoded_shape)
  autoencoder = Model(encoder_input, decoder(encoder(encoder_input)), name='autoencoder')
  return autoencoder,encoder,decoder,encoded_shape

autoencoder,encoder,decoder,encoded_shape=ConvolutionalAutoencoder(input_shape)
autoencoder.compile(loss='binary_crossentropy', optimizer='adam')

print(autoencoder.summary())
print(encoder.summary())
print(decoder.summary())

In [ ]:
# Realizar el entrenamiento

epochs=1
batch_size = 32
autoencoder.fit(x_train_noisy, x_train, validation_data=(x_test_noisy,x_test), epochs=epochs, batch_size=batch_size)

In [ ]:
# Realizar la predicción

x_decoded = autoencoder.predict(x_test_noisy)

In [ ]:
# Visualización de las imágenes originales, sus versiones ruidosas y sus respectivas reconstrucciones

num=10 # cantidad de imágenes a visualizar
print("Originales")
show_as_single_image(x_test[:num])
print("Ruidosas")
show_as_single_image(x_test_noisy[:num])
print("Restauradas")
show_as_single_image(x_decoded[:num])